<a href="https://colab.research.google.com/github/nina-prog/DataAnalysis_VAE/blob/main/VAE_v2.0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import TimeDistributed
from keras.layers import Bidirectional
from keras.layers import Dropout
from keras.layers import Reshape
from keras.utils.vis_utils import plot_model
from keras.regularizers import l2
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

# Data Preprocessing

---



## Load Data

In [2]:
### Load ecg5000 data using read_csv
ecg5000 = pd.read_csv('ECG5000_ALL.txt', sep='\s+', header=None)

In [ ]:
### Optional test and info about data set
print("Type of ecg5000: \t \t {}".format(type(ecg5000)))
print("Dimensions of ecg5000: \t \t {}".format(ecg5000.shape))
print("Number of elements of ecg5000: \t {}".format((ecg5000.size)))
print("Display first 10 rows of ecg5000: \n {}".format(ecg5000.head(10)))

In [ ]:
### Normalize dataframe with min-max-normalization to range between [-0.8, 0.8] using sklearn MinMaxScaler
min_max_scaler = MinMaxScaler(feature_range=(-0.8,0.8))
scaled_ecg5000 = pd.DataFrame(min_max_scaler.fit_transform(ecg5000))
print(scaled_ecg5000)

## Split Data

In [ ]:
### Split Data into 80/20 Training, Test
trainDF, testDF = train_test_split(scaled_ecg5000, test_size=0.2, shuffle=False)

# Get all labels from trainDF and then drop it
trainDF_Y = trainDF.iloc[:,0]
trainDF.drop(trainDF.columns[[0]], axis=1, inplace=True)

# Get all labels from testDF and then drop it
testDF_Y = testDF.iloc[:,0]
testDF.drop(testDF.columns[[0]], axis=1, inplace=True)

# Optional test and info about new data sets
print("Shape of Train DataFrame: \t {}".format(trainDF.shape))
print("Shape of Test DataFrame: \t {}".format(testDF.shape))
print("Shape of Train Y DataFrame: \t {}".format(trainDF_Y.shape))
print("Shape of Test Y DataFrame: \t {}".format(testDF_Y.shape))

## Reshape Data

In [5]:
### Convert to array
x_train = trainDF.to_numpy()
x_test = testDF.to_numpy()

y_train = trainDF_Y.to_numpy()
y_test = testDF_Y.to_numpy()

### Reshape datasets X/Y train/test into [samples, timesteps, features]
s_xtrain = len(trainDF.index) # samples
n_xtrain = len(trainDF.columns) # time steps

s_xtest = len(testDF.index) # samples
n_xtest = len(testDF.columns) # time steps

s_ytrain = len(trainDF_Y.index) # samples

s_ytest = len(testDF_Y.index) # samples

x_train = x_train.reshape(s_xtrain, n_xtrain, 1)
x_test = x_test.reshape(s_xtest, n_xtest, 1)

y_train = y_train.reshape(s_ytrain, 1, 1)
y_test = y_test.reshape(s_ytest, 1, 1)

### Properties
print("Shape of x_train: {}".format(x_train.shape))
print("Shape of x_test: {}".format(x_test.shape))

print("Shape of y_train: {}".format(y_train.shape))
print("Shape of y_test: {}".format(y_test.shape))

Shape of x_train: (4000, 140, 1)
Shape of x_test: (1000, 140, 1)
Shape of y_train: (4000, 1, 1)
Shape of y_test: (1000, 1, 1)


# Create Sample Layer

---



In [6]:
class Sampling(layers.Layer):
    """Uses (z_mean, z_log_var) to sample z"""

    def call(self, inputs):
        z_mean, z_log_var = inputs
        batch = tf.shape(z_mean)[0]
        dim = tf.shape(z_mean)[1]
        epsilon = tf.keras.backend.random_normal(shape=(batch, dim))
        return z_mean + tf.exp(0.5 * z_log_var) * epsilon

# Build Variational Autoencoder (VAE)

---



## Define Hyperparameters

In [102]:
### For better understanding visit: https://towardsdatascience.com/autoencoders-for-the-compression-of-stock-market-data-28e8c1a2da3e
### For better understanding of layers and Recreating auto encoders visit: https://machinelearningmastery.com/lstm-autoencoders/
### or for code: https://gist.github.com/GerardBCN/40349b39bc45d4550141aff6966d1619#file-stock_price_autoencoding-ipynb
### For Reshaping Issues: https://machinelearningmastery.com/reshape-input-data-long-short-term-memory-networks-keras/

### Model Properties
epochs = 12
batch_size = 16

input_shape=(None,140,1)

encoding_dim = 140
intermediate_dim = 140
latent_dim = 5            #5, because 5 class in data ecg5000
activation = 'tanh'
dropout_rate = 0.2
r_rate = 0.001


## Encoder

In [29]:
def create_encoder(encoding_dim=140, intermediate_dim=140, latent_dim = 5, activation='tanh', dropout_rate=0.2, regulazier_rate=0.001): # maybe delete encoding and latent dim, do something with input shape
    ### Define Layers
    encoder_inputs = keras.Input(shape=(140, 1), name='Encoder_Input_layer')

    encoded = Bidirectional(LSTM(intermediate_dim, activation=activation, name=''), name='Encode_1')(encoder_inputs)
    encoded = Dropout(dropout_rate, name='Dropout_1')(encoded)
    encoded = Dense(latent_dim, activation=activation, name='Encode_2', kernel_regularizer=l2(regulazier_rate), bias_regularizer=l2(regulazier_rate), activity_regularizer=l2(regulazier_rate))(encoded)

    z_mean = Dense(latent_dim, activation="softplus", name="z_mean")(encoded) 
    z_log_var = Dense(latent_dim, activation="softplus", name="z_log_var")(encoded)

    z = Sampling(name='Sample_layer')([z_mean, z_log_var])

    ### Instantiate encoder
    encoder = keras.Model(encoder_inputs, [z_mean, z_log_var, z], name="encoder")

    ### Configure encoder for training
    encoder.compile(optimizer='adam', loss='mean_squared_error') # necessary? because we define train step in vae model

    return encoder

In [ ]:
### Build Encoder # or only do create model later on
encoder = create_encoder() 
encoder.summary()
plot_model(encoder, show_shapes=True, to_file='reconstruct_lstm_encoder.png')

## Decoder

In [30]:
def create_decoder(encoding_dim=140, intermediate_dim=140, latent_dim = 5, activation='tanh', dropout_rate=0.2, regulazier_rate=0.001): # maybe delete encoding and latent dim, do something with input shape
    ### Define Layers
    latent_inputs = keras.Input(shape=(latent_dim,), name='Decoder_Input_layer')

    decoded = Dense(encoding_dim, activation=activation, name='Decode_1', kernel_regularizer=l2(regulazier_rate), bias_regularizer=l2(regulazier_rate), activity_regularizer=l2(regulazier_rate))(latent_inputs)
    decoded = Reshape((140,1), name='Decode_2')(decoded)
    decoded = Dropout(dropout_rate, name='Dropout_1')(decoded)
    decoded = Bidirectional(LSTM(intermediate_dim, activation=activation, return_sequences=True, name=''), name='Decode_3')(decoded)

    decoder_outputs = TimeDistributed(Dense(1, activation=activation, name=''),name='Decoder_Output_Layer')(decoded)

    ### Instantiate decoder
    decoder = keras.Model(latent_inputs, decoder_outputs, name="decoder")

    ### Configure encoder for training
    decoder.compile(optimizer='adam', loss='mean_squared_error') # necessary? because we define train step in vae model

    return decoder

In [ ]:
### Build Decoder # or only do create model later on
decoder = create_decoder()
decoder.summary()
plot_model(decoder, show_shapes=True, to_file='reconstruct_lstm_decoder.png')

## VAE

Define VAE Model

In [39]:
class VAE(keras.Model):
    """Combines the encoder and decoder into an end-to-end model for training."""

    def __init__(self, encoder, decoder, **kwargs):
        super(VAE, self).__init__(**kwargs)
        self.encoder = encoder
        self.decoder = decoder

    @property
    def metrics(self):
        return [
            self.total_loss_tracker,
            self.reconstruction_loss_tracker,
            self.kl_loss_tracker,
        ]

    def train_step(self, data):
        if isinstance(data, tuple):
            data = data[0]
        with tf.GradientTape() as tape:
            z_mean, z_log_var, z = encoder(data)
            reconstruction = decoder(z)
            # reconstruction_loss = distance between Input and Output
            reconstruction_loss = tf.reduce_mean(
                #Alternative: keras.losses.binary_crossentropy(data, reconstruction)
                keras.losses.mean_squared_error(data, reconstruction)
            )
            # kl_loss = distance between distributions and thus ensures the regular laten space
            kl_loss = 1 + z_log_var - tf.square(z_mean) - tf.exp(z_log_var)
            kl_loss = tf.reduce_mean(kl_loss)
            kl_loss *= -0.5
            total_loss = reconstruction_loss + kl_loss
        grads = tape.gradient(total_loss, self.trainable_weights)
        self.optimizer.apply_gradients(zip(grads, self.trainable_weights))
        self.total_loss_tracker.update_state(total_loss) #new
        self.reconstruction_loss_tracker.update_state(reconstruction_loss) #new
        self.kl_loss_tracker.update_state(kl_loss) #new
        return {
            "loss": self.total_loss_tracker.result(), #total_loss,
            "reconstruction_loss": self.reconstruction_loss_tracker.result(), #reconstruction_loss,
            "kl_loss":  self.kl_loss_tracker.result(),#kl_loss,
        }

    def test_step(self, data):
        # Unpack the data
        x, y = data
        # Compute predictions
        y_pred = self(x, training=False)
        # Updates the metrics tracking the loss
        self.compiled_loss(y, y_pred, regularization_losses=self.losses)
        # Update the metrics.
        self.compiled_metrics.update_state(y, y_pred)
        # Return a dict mapping metric names to current value.
        # Note that it will include the loss (tracked in self.metrics).
        return {m.name: m.result() for m in self.metrics}

    def call(self, data):
        z_mean, z_log_var, z = self.encoder(data)
        reconstructed = self.decoder(z)
        return reconstructed

Build VAE connecting Encoder and Decoder

In [31]:
### Function to create model, required for KerasClassifier
def create_model(name, activation='tanh', intermediate_dim=140, dropout_rate=0.2, regulazier_rate=0.2, optimizer='Adam', learn_rate='0.001', momentum=0):
    # create encoder 
    encoder = create_encoder(activation=activation, intermediate_dim=intermediate_dim, dropout_rate=dropout_rate, regulazier_rate=regulazier_rate)
    # create decoder 
    decoder = create_decoder(activation=activation, intermediate_dim=intermediate_dim, dropout_rate=dropout_rate, regulazier_rate=regulazier_rate)
    # create vae
    vae = VAE(encoder, decoder, name=name)
    # compile model
    opt = optimizer(lr=learn_rate, momentum=momentum)
    vae.compile(optimizer=opt)
    return vae # possible to do encoder, decoder, vae?

In [32]:
#vae = VAE(encoder, decoder, name="VAE") # not necessary when using create_model()
vae = create_model(name="VAE")
vae.build(input_shape)
# vae.encoder.summary()
# plot_model(encoder, show_shapes=True, to_file='reconstruct_lstm_encoder.png')
# vae.decoder.summary()
# plot_model(decoder, show_shapes=True, to_file='reconstruct_lstm_decoder.png')
vae.summary(line_length=100)
plot_model(vae, show_shapes=True, to_file='reconstruct_lstm_variational_autoencoder.png') # does not work correctly

NameError: ignored

In [ ]:
### Instantiate VAE model
# The output of vae model is the output of decoder in which its input is taken from the output of encoder !
#decoder_outputs = decoder(encoder(encoder_inputs)[2])
#vae = keras.Model(encoder_inputs, decoder_outputs, name='vae_v2_0')

#vae.summary()
#plot_model(vae, show_shapes=True, to_file='reconstruct_lstm_variational_autoencoder.png', rankdir='LR')

# Train VAE

---



## Train

In [48]:
### To improve training see here: https://becominghuman.ai/using-variational-autoencoder-vae-to-generate-new-images-14328877e88d
### Train
vae.compile(optimizer='Adam') #, loss='mean_squared_error' # not necessary when using create_model() earlier, because that functon already does compile
history = vae.fit(x_train, x_train, epochs=epochs, batch_size=batch_size, validation_data=(x_test, x_test))

Epoch 1/12
250/250 [==============================] - 105s 386ms/step - loss: 0.2041 - reconstruction_loss: 0.0308 - kl_loss: 0.1732 - val_loss: 0.0367
Epoch 2/12
250/250 [==============================] - 96s 383ms/step - loss: 0.0572 - reconstruction_loss: 0.0251 - kl_loss: 0.0321 - val_loss: 0.0358
Epoch 3/12
250/250 [==============================] - 96s 384ms/step - loss: 0.0374 - reconstruction_loss: 0.0239 - kl_loss: 0.0136 - val_loss: 0.0356
Epoch 4/12
250/250 [==============================] - 96s 382ms/step - loss: 0.0311 - reconstruction_loss: 0.0234 - kl_loss: 0.0077 - val_loss: 0.0358
Epoch 5/12
250/250 [==============================] - 96s 382ms/step - loss: 0.0281 - reconstruction_loss: 0.0231 - kl_loss: 0.0050 - val_loss: 0.0361
Epoch 6/12
250/250 [==============================] - 96s 385ms/step - loss: 0.0263 - reconstruction_loss: 0.0228 - kl_loss: 0.0035 - val_loss: 0.0364
Epoch 7/12
250/250 [==============================] - 96s 384ms/step - loss: 0.0252 - reconst

## Recreate Latent Space

In [ ]:
# Encoder output is a list [z_mean, z_log_var, z] thus list[2] = z, see subsection encoder line 12

### Extract myu i.e. z_mean
z_mean = encoder.predict(x_test)[0]
print("----- z_mean: -----")
print(z_mean)
print("\n")

### Extract sigma i.e. z_log_var
z_log_var = encoder.predict(x_test)[1]
print("----- z_log_var: -----")
print(z_log_var)
print("\n")

### Extract z_values and predict x_test
z_values = encoder.predict(x_test)[2]
# decoded_ecg5000 = vae.predict(x_test)
decoded_ecg5000 = decoder.predict(z_values)

# z_values contains list of each z_value per sample, i.e. we get 1000 SubLists with 5 elements in each.
# Those 5 elements (z_values for Sample i) is our bottleneck which the decoder
# recieves.
print("----- z_values: -----")
print(z_values)
print("\n")

### Properties
print("Shape of z_mean: {}".format(z_mean.shape))
print("Shape of z_log_var: {}".format(z_log_var.shape))
print("Shape of decoded_ecg5000: {}".format(decoded_ecg5000.shape))
print("Shape of z_values: {}".format(z_values.shape))

## Display the training progress

In [ ]:
### Loss vs Reconstruction_loss
plt.figure(figsize=(16,8))
plt.plot(history.history['loss'])
plt.plot(history.history['reconstruction_loss'])
plt.legend(["Loss", "Reconstruction Loss"])
plt.xlabel("Epoch")
plt.title("Loss vs. Reconstruction Loss")

In [ ]:
### Train loss vs val loss
#Returns the loss value & metrics values for the model in test mode

plt.figure(figsize=(16,8))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.legend(["Loss", "Validation Loss"])
plt.xlabel("Epoch")
plt.title("Loss vs. Validation Loss")

In [ ]:
#########################################################To-Do
### Latent space
# x,y Plot
plt.figure(figsize=(15,6), linewidth=1)
plt.plot(z_values[:,0], 'o')
plt.plot(z_values[:,1], '+')
plt.show()

# Scatterplot
plt.figure(figsize=(15,12))
plt.scatter(z_values[:,0], z_values[:,1], s=80, c=y_test, cmap='viridis') # or cmap=hsv
plt.colorbar()
plt.grid()
plt.show()

# Plot Data Results

---



In [87]:
### Test if Input fits Dim of Output
print("Shape of Input x_train: {}".format(x_train.shape))
print("Shape of Output x_hat_train: {}".format(decoded_ecg5000.shape))

### Covert to 2D Array -- ("-1" = make a dimension (here rows) the size that will use the remaining unspecified elements)
new_x_train= x_train.reshape(-1,140)
new_decoded_ecg5000 = decoded_ecg5000.reshape(-1,140)

print("Shape of Input after reshaping: {}".format(new_x_train.shape))
print("Shape of Output after reshaping: {}".format(new_decoded_ecg5000.shape))

Shape of Input x_train: (4000, 140, 1)
Shape of Output x_hat_train: (1000, 140, 1)
Shape of Input after reshaping: (4000, 140)
Shape of Output after reshaping: (1000, 140)


In [ ]:
### Plot only one sample
i = 934 # indize/sample which is going to be plotted
plt.figure(linewidth = 1, figsize=(25,6))
plt.title('Autoencoder Result')
plt.xlabel('time steps')
plt.plot(new_x_train[i], label='original ecg5000')
plt.plot(new_decoded_ecg5000[i], label='decoded ecg5000')
plt.legend(loc="upper left")
plt.show()

In [ ]:
### Plot Multiple Samples
n_rows = 2                   
n_cols = 2

# Size Properties and layout design for tighter representation
fig, axs = plt.subplots(nrows = n_rows, ncols = n_cols, figsize=(23,20))
fig.tight_layout(w_pad=4, h_pad = 5)

# Subplotting
i = 100
for row in range(n_rows):
  for col in range(n_cols):
    axs[row, col].plot(new_x_train[i])
    axs[row, col].plot(new_decoded_ecg5000[i])
    axs[row, col].legend(["Original ECG5000 Sample {}".format(i), "Decoded ECG5000 Sample {}".format(i)])
    axs[row, col].set(xlabel = "Time Steps", ylabel = "Heartbeat Interpolated", title = "Sample {}".format(i))
    i = i + 200


# Optimization

---

## Hyperparameter (Sckit_GridSearchCV)

In [94]:
from sklearn.model_selection import GridSearchCV
from keras.wrappers.scikit_learn import KerasClassifier

In [96]:
### To understand it better see here: https://mlfromscratch.com/gridsearch-keras-sklearn/#/
### Define Function to gridSearch
def gridSearch_pipeline(X_train_data, X_test_data, y_train_data, y_test_data, model, param_grid, cv=10, scoring_fit='neg_mean_squared_error', do_probabilities = False):
    # define gridSearch
    gs = GridSearchCV(
        estimator=model,
        param_grid=param_grid, 
        cv=cv, 
        n_jobs=1, 
        scoring=scoring_fit,
        verbose=2
    )
    # fit model
    fitted_model = gs.fit(X_train_data, y_train_data)
    # get all! rsults in a dataframe 
    df_result = pd.DataFrame(gs.cv_results_)
    
    if do_probabilities:
      pred = fitted_model.predict_proba(X_test_data)
    else:
      pred = fitted_model.predict(X_test_data)
    
    return fitted_model, pred, df_result

In [ ]:
### Define evaluated params and it's value range
param_grid = {
              'epochs' :              [30, 50, 100],
              'batch_size' :          [16, 32],
              'activation' :          ['tanh', 'relu', 'sigmoid'],
              'dropout_rate' :        [0.0, 0.2, 0.3],
              'regulazier_rate' :     [0.001,0.004],
              'optimizer' :           ['Adam', 'SGD'],
              'learn_rate' :          [0.0, 0.001, 0.01, 0.1, 0.2, 0.3],
              'momentum' :            [0.0, 0.2, 0.4, 0.6, 0.8, 0.9] 
             }
#learning_rate?, kernel_regularizer?, bias_regularizer?, activity_regularizer?
################ everthing except batch_size and epochs need to be naded as a parameter in create_model()
################ grid param need to be parameters of create model? or have to be initiated in the model? dunno

### Wrap keras custom VAE model with the KerasClassifier so it can be used in scikit-learn
# see here how it is wrapped: https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/
model = KerasClassifier(build_fn=create_model())

### Run GridSearch
# see here to solve error: https://stackoverflow.com/questions/23866833/whats-the-full-specification-for-implementing-a-custom-scikit-learn-estimator
model, pred, df_GSresult = gridSearch_pipeline(x_train, x_test, y_train, y_test, model, param_grid, cv=5, scoring_fit='neg_log_loss')

### Summarize results
print("Best: %f using %s" % (model.best_score_, model.best_params_))
print(df_GSresult)

## Dropout

In [ ]:
###Dropout_rate

# configure the experiment
def experiment_dropout():
  # configure the experiment
  n_dropout = [0.0, 0.2, 0.4, 0.6, 0.8]
  # run the experiment
  results = []
  for drop_value in n_dropout:
      # set dropout
      drop_out_rate = drop_value
      print("----- Dropout Rate: {} -----".format(drop_out_rate))
      # evaluate
      # rather shorten code with defining a train function of code above and using it here
      vae = VAE(encoder, decoder, name="VAE")
      vae.compile(optimizer='adam', loss='mean_squared_error')
      history = vae.fit(x_train, y_train, epochs=epochs, batch_size=batch_size, validation_data=(x_test, y_test), verbose=0)
      # report performance
      # rathr make a dataframe or something different which is simpler to plot
      evaluation = []
      evaluation.append(vae.evaluate(x_test, y_test))
      evaluation.append(drop_value)

      res = []
      res.append(history.history["val_loss"])
      print("val_loss = {}".format(res))
      results.append(evaluation)
  return results

In [ ]:
results = experiment_dropout()
# summarize results
print(results)